In [ ]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib import gridspec


from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Descriptors3D, Draw, rdMolDescriptors

#from Bio import pairwise2
#from Bio.pairwise2 import format_alignment
#import itertools
#import mmtf

#from scipy.cluster.hierarchy import dendrogram, linkage
import progressbar
from math import pi

In [ ]:
table=pd.read_excel('Cys_DataBase_FINAL.xlsx')

In [ ]:
table.head(2)

In [ ]:
bar = indexgressbar.ProgressBar(max_value=len(table.index))

In [ ]:
for name in table.index:
    for file in os.listdir('DATA/Interactions/'):
        if table.loc[name,'identifier'].split('_')[0]==file.split('.')[0]:
            with open('DATA/Interactions/'+file) as json_file:
                data = json.load(json_file)
                table.loc[name,'hbond']=len(data.get('Interactions')['hbond'])
                table.loc[name,'hdon_pi']=len(data.get('Interactions')['hdon_pi'])
                table.loc[name,'ionic']=len(data.get('Interactions')['ionic'])
                table.loc[name,'pi_pi']=len(data.get('Interactions')['pi_pi'])
                table.loc[name,'poor_ang']=len(data.get('Interactions')['poor_ang'])
                table.loc[name,'unclass']=len(data.get('Interactions')['unclass'])
                table.loc[name,'unfav']=len(data.get('Interactions')['unfav'])
                table.loc[name,'vdW']=len(data.get('Interactions')['vdW'])
                table.loc[name,'scorpion_score']=float(data.get('Scorpion_score'))
                table.loc[name,'smiles_cov']=data.get('Structures')['smiles']
                bar.update(name)

In [ ]:
for name in table.index:
    try:
        x=mmtf.fetch(table.loc[name,'identifier'].split('_')[0])
        table.loc[name,'sequence']=x.entity_list[0]['sequence']
        bar.update(name)
    except Exception:
        continue

In [ ]:
table.head(2)

In [ ]:
table.to_excel('Cys_DataBase_FINAL.xlsx')

# SMILES and analysis of molecular descriptor (CHEM_3 enviroment)

In [ ]:
mols=Chem.SDMolSupplier('DATA/final.sdf',False)

In [ ]:
for name in table.index:
    for i,file in enumerate(os.listdir('DATA/Ligands/')):
        if table.loc[name,'identifier'].split('_')[0]==file.split('.')[0]:
            
            mol=mols[i]
            mol.UpdatePropertyCache(strict=False)
            Chem.SanitizeMol(mol,Chem.SanitizeFlags.SANITIZE_FINDRADICALS|Chem.SanitizeFlags.SANITIZE_KEKULIZE
                             |Chem.SanitizeFlags.SANITIZE_SETAROMATICITY|Chem.SanitizeFlags.SANITIZE_SETCONJUGATION
                             |Chem.SanitizeFlags.SANITIZE_SETHYBRIDIZATION|Chem.SanitizeFlags.SANITIZE_SYMMRINGS,catchErrors=True)
            
            table.loc[name,'MolWt_cov']=Descriptors.MolWt(mol)
            table.loc[name,'LogP_cov']=Descriptors.MolLogP(mol)
            table.loc[name,'NumHAcceptors_cov']=Descriptors.NumHAcceptors(mol)
            table.loc[name,'NumHDonors_cov']=Descriptors.NumHDonors(mol)
            table.loc[name,'NumHeteroatoms_cov']=Descriptors.NumHeteroatoms(mol)
            table.loc[name,'NumRotableBonds_cov']=Descriptors.NumRotatableBonds(mol)

            table.loc[name,'TPSA_cov']=Descriptors.TPSA(mol)
            table.loc[name,'PMI1_cov']=Descriptors3D.PMI1(mol)
            table.loc[name,'PMI2_cov']=Descriptors3D.PMI2(mol)
            table.loc[name,'PMI3_cov']=Descriptors3D.PMI3(mol)
            table.loc[name,'PBF_cov']=rdMolDescriptors.CalcPBF(mol)
            table.loc[name,'NPR1_cov']=rdMolDescriptors.CalcNPR1(mol)
            table.loc[name,'NPR2_cov']=rdMolDescriptors.CalcNPR2(mol)
            table.loc[name,'ISF_cov']=Descriptors3D.InertialShapeFactor(mol)

In [ ]:
table.head(2)

In [ ]:
table.to_excel('Cys_DataBase_FINAL.xlsx')

# Charts

In [ ]:
size=len(table['sequence'].dropna())
hmap=np.empty(shape=(size,size))
bar = progressbar.ProgressBar(max_value=len(table['sequence'].dropna()))
hmap_table=pd.DataFrame()
for index,i in enumerate(table['sequence'].dropna()):
    for jndex,j in enumerate(table['sequence'].dropna()):
        a=i
        b=j
        alignment= pairwise2.align.globalxx(a, b,score_only=True)
        identity=((alignment*100)/len(b))
        hmap[index,jndex]=identity
        hmap_table.loc[table.loc[index,'identifier'].split('_')[0],table.loc[jndex,'identifier'].split('_')[0]]=identity
    bar.update(index)

In [ ]:
linked = linkage(hmap, 'single')
labelList =[i for i in hmap_table.index]

In [ ]:
plt.figure(figsize=(10,10))

ax1=plt.subplot()
o=dendrogram(linked,  
            orientation='left',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)

ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

plt.tick_params ('both',width=2,labelsize=5)
plt.tight_layout()
plt.savefig('DATA/Charts/HCL.svg',dpi=600,format='svg',transparent=False)
plt.show()

In [ ]:
new_data=list(reversed(o['ivl']))
# we create a new table with the order of HCL
hmap_2=np.empty(shape=(size,size))
for index,i in enumerate(new_data):
    for jndex,j in enumerate(new_data):
        hmap_2[index,jndex]=hmap_table.loc[i].at[j]

In [ ]:
figure= plt.figure(figsize=(30,30))
gs1 = gridspec.GridSpec(2,7)
gs1.update(wspace=0.01)
ax1 = plt.subplot(gs1[0:-1, :2])
dendrogram(linked, orientation='left', distance_sort='descending',show_leaf_counts=True,no_labels=True)
ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = plt.subplot(gs1[0:-1,2:6])
f=ax2.imshow (hmap_2, cmap='RdBu_r', interpolation='nearest')

ax2.set_title('Sequence Indentity',fontsize=30,weight='bold')
ax2.set_xticks (range(len(new_data)))
ax2.set_yticks (range(len(new_data)))
ax2.set_xticklabels (new_data,rotation=90,size=3)
ax2.set_yticklabels (new_data,size=3)

ax3 = plt.subplot(gs1[0:-1,6:7])
m=plt.colorbar(f,cax=ax3,shrink=0.75,orientation='vertical',spacing='uniform',pad=0.01)
m.set_label ('% Indentity',fontsize=25)
m.ax.tick_params(labelsize=25)
plt.savefig('DATA/Charts/HCL_Hmap.svg',dpi=600,format='svg',transparent=False)
plt.tick_params ('both',width=2)
plt.plot()

In [ ]:
hmap_table.to_excel('identity.xlsx')

## Pie chart

In [ ]:
x=table.groupby('Covalent_reaction_type').size().plot(kind='pie')

In [ ]:
df = pd.DataFrame(table.groupby('Covalent_reaction_type').size())

In [ ]:
df.index

In [ ]:
df=df.sort_values(0,ascending=False)

In [ ]:
df.to_excel('Table_reaction_type.xlsx')

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6), subplot_kw=dict(aspect="equal"))


wedges, texts,autotexts =ax1.pie(df,wedgeprops=dict(width=0.3), startangle=-40,autopct='%1.1f%%',pctdistance=0.82)

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.5)

kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2.+ p.theta1
    
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax1.annotate(df.index[i], xy=(x, y), xytext=(1.2*np.sign(x), 1.4*y),
                 horizontalalignment=horizontalalignment, **kw)

ax1.axis('equal')  

plt.tight_layout()

plt.savefig('DATA/Charts/Pie_reaction_type_2.svg',dpi=600,format='svg',transparent=False)
plt.show()

In [ ]:
from math import log10
from matplotlib import cm

#number of data points
n = len(df.index)
#find max value for full ring
k = 10 ** int(log10(max(df[0])))
m = k * (1 + max(df[0]) // k)

#radius of donut chart
r = 1.5
#calculate width of each ring
w = r / n 

#create colors along a chosen colormap
colors = [cm.terrain(i / n) for i in range(n)]

#create figure, axis
fig, ax = plt.subplots(figsize=(10,10))
ax.axis("equal")

#create rings of donut chart
for i in range(n):
    #hide labels in segments with textprops: alpha = 0 - transparent, alpha = 1 - visible
    innerring, _ = ax.pie([m - df[0][i], df[0][i]], radius = r - i * w, startangle = 90, labels = ["", df.index[i]], labeldistance = 1 - 1 / (1.5 * (n - i)), textprops = {"alpha": 0}, colors = ["white", colors[i]])
    plt.setp(innerring, width = w, edgecolor = "white")

plt.legend(loc='lower center',frameon=False)
plt.savefig('DATA/Charts/Pie_reaction_type_3.svg',dpi=600,format='svg',transparent=False)
plt.show()

## Pie plot organism

In [ ]:
table['organism']=[str(i).lower() for i in table['organism']]

In [ ]:
table['organism']

In [ ]:
x=table.groupby('organism').size()

In [ ]:
x

In [ ]:
names=table['organism'].dropna().unique()

In [ ]:
plt.figure(figsize=(10,10))

ax1=plt.subplot()

ax1=table.groupby('Organism').size().plot(kind='pie')

ax1.axis('equal')  
plt.tight_layout()
#plt.savefig('DATA/Charts/Pie_organism.svg',dpi=600,format='svg',transparent=False)
plt.show()

## MolWt distribution

In [ ]:
table.columns

In [ ]:
table['MolWt_cov'].dropna()

In [ ]:
from scipy import stats ## Ver después en el notebook de Statmodels
h=list(table['MolWt_cov'].dropna()) ## Disperción en X
h.sort()
hmean = np.mean(h)
hstd = np.std(h)
pdf = stats.norm.pdf(h, hmean, hstd)

In [ ]:
plt.figure(figsize=(8,6))
plt.tick_params ('both',width=2,labelsize=12)
ax = plt.subplot(111)
ax.plot(h,pdf,'k--',linewidth=2.0)
ax.hist(h,density=True,histtype='step',color='gray',linewidth=1.5)
ax.set_title ('Probability Distribution',fontsize=14,fontweight='bold')
ax.set_xlabel ('Molecular Weight',fontsize=12,fontweight='bold')
ax.set_ylabel ('Probability',fontsize=12,fontweight='bold')

## Añade una linea en el promedio de los datos
ax.axvline(x=hmean,color='r',linewidth=1,linestyle='--',)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('DATA/Charts/MolWt.svg',dpi=600,format='svg',transparent=False)
plt.show()

In [ ]:
h=list(table['LogP_cov'].dropna()) ## Disperción en X
h.sort()
hmean = np.mean(h)
hstd = np.std(h)
pdf = stats.norm.pdf(h, hmean, hstd)

plt.figure(figsize=(8,6))
plt.tick_params ('both',width=2,labelsize=12)
ax = plt.subplot(111)
ax.plot(h,pdf,'k--',linewidth=2.0)
ax.hist(h,density=True,histtype='step',color='gray',linewidth=1.5)
ax.set_title ('Probability Distribution',fontsize=14,fontweight='bold')
ax.set_xlabel ('LogP',fontsize=12,fontweight='bold')
ax.set_ylabel ('Probability',fontsize=12,fontweight='bold')

## Añade una linea en el promedio de los datos
ax.axvline(x=hmean,color='r',linewidth=1,linestyle='--',)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('DATA/Charts/LogP.svg',dpi=600,format='svg',transparent=False)
plt.show()

In [ ]:
h=list(table['TPSA_cov'].dropna()) ## Disperción en X
h.sort()
hmean = np.mean(h)
hstd = np.std(h)
pdf = stats.norm.pdf(h, hmean, hstd)

plt.figure(figsize=(8,6))
plt.tick_params ('both',width=2,labelsize=12)
ax = plt.subplot(111)
ax.plot(h,pdf,'k--',linewidth=2.0)
ax.hist(h,density=True,histtype='step',color='gray',linewidth=1.5)
ax.set_title ('Probability Distribution',fontsize=14,fontweight='bold')
ax.set_xlabel ('TPSA',fontsize=12,fontweight='bold')
ax.set_ylabel ('Probability',fontsize=12,fontweight='bold')

## Añade una linea en el promedio de los datos
#ax.axvline(x=hmean,color='r',linewidth=1,linestyle='--',)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('DATA/Charts/TPSA.svg',dpi=600,format='svg',transparent=False)
plt.show()

In [ ]:
h=list(table['scorpion_score'].dropna()) ## Disperción en X
h.sort()
hmean = np.mean(h)
hstd = np.std(h)
pdf = stats.norm.pdf(h, hmean, hstd)

plt.figure(figsize=(8,6))
plt.tick_params ('both',width=2,labelsize=12)
ax = plt.subplot(111)
ax.plot(h,pdf,'k--',linewidth=2.0)
ax.hist(h,density=True,histtype='step',color='gray',linewidth=1.5)
ax.set_title ('Probability Distribution',fontsize=14,fontweight='bold')
ax.set_xlabel ('Scorpion score',fontsize=12,fontweight='bold')
ax.set_ylabel ('Probability',fontsize=12,fontweight='bold')

## Añade una linea en el promedio de los datos
ax.axvline(x=hmean,color='r',linewidth=1,linestyle='--',)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.savefig('DATA/Charts/Scorpion_score.svg',dpi=600,format='svg',transparent=False)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.tick_params ('both',width=2,labelsize=12)

ax = plt.subplot(111)
ax.scatter(table['NPR1_cov'].dropna(),table['NPR2_cov'].dropna(),color='C3',s=14)

x1, y1 = [0.5, 0], [0.5, 1]
x2, y2 = [0.5, 1], [0.5, 1]
x3, y3 = [0,1],[1,1]
plt.plot(x1, y1,x2,y2,x3,y3,c='gray',ls='--',lw=2)

plt.xlabel ('NPR1',fontsize=20,fontweight='bold')
plt.ylabel ('NPR2',fontsize=20,fontweight='bold')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.text(0, 1.01,s='Rod',fontsize=14,horizontalalignment='center',verticalalignment='center',fontweight='bold')
plt.text(1, 1.01,s='Sphere',fontsize=14,horizontalalignment='center',verticalalignment='center',fontweight='bold')
plt.text(0.5, 0.49,s='Disc',fontsize=14,horizontalalignment='center',verticalalignment='center',fontweight='bold')

plt.tick_params ('both',width=2,labelsize=16)
plt.tight_layout()
plt.savefig('DATA/Charts/NPR.svg',dpi=600,queality=95,format='svg')
plt.show()

## Spider plots

In [ ]:
table.columns

In [ ]:
table=table.drop(columns=['NumHeteroatoms_cov'])

In [ ]:
table['MolWt_cov']=[i/500 for i in table['MolWt_cov']]
table['LogP_cov']=[i/5 for i in table['LogP_cov']]
table['NumHAcceptors_cov']=[i/10 for i in table['NumHAcceptors_cov']]
table['NumHDonors_cov']=[i/5 for i in table['NumHDonors_cov']]
table['NumRotableBonds_cov']=[i/10 for i in table['NumRotableBonds_cov']]
table['TPSA_cov']=[i/140 for i in table['TPSA_cov']]

In [ ]:
table.groupby('Covalent_reaction_type').groups.keys()

In [ ]:
r=table.groupby('Covalent_reaction_type').get_group('1,2-Addition to sp2 Atom')

In [ ]:
r.columns[-14:-8]

In [ ]:
categories=list(table.columns[-14:-8])
N = len(categories)

In [ ]:
RoF=[1,1,1,1,1,1,1]

In [ ]:
values=r[categories].values[0]
values=np.append(values,values[:1])

In [ ]:
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

In [ ]:
fig=plt.figure(figsize=(8,8))

ax = fig.add_axes([1, 1, 1, 1],projection='polar')

ax.set_title('Thioether formation',size='25',weight='bold')

plt.xticks(angles, [i.split('_')[0] for i in categories],color='k', size=18,ha='center',va='top')

plt.tick_params(axis='y',width=2,labelsize=16, grid_alpha=0.15)

ax.set_rlabel_position(0)

for i in range(len(r.index)):
    values=r[categories].values[i]
    values=np.append(values,values[:1])
    ax.plot(angles, values, linewidth=1 ,color='k', linestyle='-',alpha=0.2)
    ax.fill(angles, values, 'C2', alpha=0.025)

ax.plot(angles, RoF, linewidth=3, linestyle='solid',color='red')
#ax.fill(angles, RoF, 'red', alpha=0.2)
plt.savefig('DATA/Charts/Radar_charts/thioether formation.svg',dpi=600,queality=95,format='svg')
plt.plot()

## PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

In [ ]:
table.columns

In [ ]:
descriptors=table.columns[22:39]

In [ ]:
table_nona=table[descriptors].dropna()

In [ ]:
for i in descriptors:
    [float(i) for i in table_nona[i]]

In [ ]:
table_nona=table_nona.apply(pd.to_numeric)